In [1]:
# importing required modules
import PyPDF2
import tabula
from tabula.io import read_pdf
from pprint import pprint
import pandas as pd
from pandas import json_normalize
import numpy as np 
import json
import glob
import os

In [2]:
mes= 'Jun'
u= 'Leja'

In [3]:
# set the directory path
path = f'C://Users//{u}//Downloads//MVR//national//{mes}/'
# loop through the directory
# create a list of all the xlsx files in the folder
files = os.listdir(path)

# loop through each file in the list
for file in files:
    # check if the file is an xlsx file
    if file.endswith(".xlsx"):
        
        # read the xlsx file into a dataframe
        df = pd.read_excel(path + file, sheet_name='Adjustments')
    
        new_filename = file.split('.')[0] + '.csv'
        # save the dataframe as a csv file with the same name as the xlsx file
        df.to_csv(path +  new_filename, index=False, date_format='%m/%d/%Y')

In [4]:
all_data = pd.DataFrame()

for f in glob.glob(f"C:/Users/{u}/Downloads/MVR/national/{mes}/*.csv"):
    df = pd.read_csv(f)
    all_data = all_data.append(df,ignore_index=True)

df= all_data.fillna('')



df.info()
print(df)
#df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Gov State          521 non-null    object
 1   Product            521 non-null    object
 2   Quote Num          521 non-null    object
 3   Drivers Name       521 non-null    object
 4   Drivers License #  521 non-null    object
 5   DL State           521 non-null    object
 6   Order Date         521 non-null    object
 7   TransType          521 non-null    object
 8   Amount             521 non-null    object
 9   Agent              521 non-null    object
 10  Quoting Producer   521 non-null    object
dtypes: object(11)
memory usage: 44.9+ KB
    Gov State Product   Quote Num                      Drivers Name  \
0          NC     PPA  98341425.0          Nicole Calihua Hernandez   
1          NC     PPA  98340786.0          Anderson Menendez Campos   
2          NC     PPA  982955

In [5]:
#df=df[df['Quoting Producer' ]!=""].dropna(axis = 1, how = 'any') #se queda sólo con los que cruzaron

df=df[df['Quote Num']!=""] #se queda sólo con los que cruzaron

df['Marca'] = df.index

df['Amount']= pd.to_numeric(df['Amount']) * -1


lhc=df[df['TransType']!='Viol Hist Chargeback']

vhc= df[df['TransType']=='Viol Hist Chargeback']

#new_header = lhc.iloc[0] #grab the first row for the header
#lhc = lhc[1:] #take the data less the header row
#lhc.columns = new_header #set the header row as the df heade


In [6]:
lhc.info()
vhc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 0 to 514
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          329 non-null    object 
 1   Product            329 non-null    object 
 2   Quote Num          329 non-null    object 
 3   Drivers Name       329 non-null    object 
 4   Drivers License #  329 non-null    object 
 5   DL State           329 non-null    object 
 6   Order Date         329 non-null    object 
 7   TransType          329 non-null    object 
 8   Amount             329 non-null    float64
 9   Agent              329 non-null    object 
 10  Quoting Producer   329 non-null    object 
 11  Marca              329 non-null    int64  
dtypes: float64(1), int64(1), object(10)
memory usage: 33.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 185 entries, 3 to 518
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  -

In [7]:
lhc['nombres'] = lhc['Drivers Name'].apply(lambda x: len(str(x).split(' ')))

lhc['Nombre'] = lhc['Drivers Name'].apply(lambda s: s.split(' ')[0])

C:\Users\Leja\AppData\Local\Temp/ipykernel_18304/4140874511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lhc['nombres'] = lhc['Drivers Name'].apply(lambda x: len(str(x).split(' ')))
C:\Users\Leja\AppData\Local\Temp/ipykernel_18304/4140874511.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lhc['Nombre'] = lhc['Drivers Name'].apply(lambda s: s.split(' ')[0])


In [8]:
lhc['Apell'] = np.where(lhc['nombres'] == 2, lhc['Drivers Name'].str.split(" ").str[1:].str.join(" "), 
                        np.where(lhc['nombres'] == 3, lhc['Drivers Name'].str.split(" ").str[1:2].str.join(" "),
                                     lhc['Drivers Name'].str.split(" ").str[2:4].str.join(" ")))
#"-".join(a.split("-", 2)[:2])
#

lhc['nk'] = (lhc['Nombre'].str[:5] + lhc['Apell'].str[:5]).str.upper()

#lhc = lhc.reindex


lhc['Marca'] = lhc.index

lhc['Name'] = lhc['Drivers Name'].str.upper()

#lhc['Amount']= pd.to_numeric(lhc['Amount']) * -1

lhc['Drivers Name']= lhc['Drivers Name'].str.upper()

lhc.info()
#lhc.dtypes


#lhc.to_excel(r'C:\Users\Leja\Downloads\NG.xlsx', engine='xlsxwriter')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 0 to 514
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          329 non-null    object 
 1   Product            329 non-null    object 
 2   Quote Num          329 non-null    object 
 3   Drivers Name       329 non-null    object 
 4   Drivers License #  329 non-null    object 
 5   DL State           329 non-null    object 
 6   Order Date         329 non-null    object 
 7   TransType          329 non-null    object 
 8   Amount             329 non-null    float64
 9   Agent              329 non-null    object 
 10  Quoting Producer   329 non-null    object 
 11  Marca              329 non-null    int64  
 12  nombres            329 non-null    int64  
 13  Nombre             329 non-null    object 
 14  Apell              329 non-null    object 
 15  nk                 329 non-null    object 
 16  Name               329 non

C:\Users\Leja\AppData\Local\Temp/ipykernel_18304/3476419925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lhc['Apell'] = np.where(lhc['nombres'] == 2, lhc['Drivers Name'].str.split(" ").str[1:].str.join(" "),
C:\Users\Leja\AppData\Local\Temp/ipykernel_18304/3476419925.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lhc['nk'] = (lhc['Nombre'].str[:5] + lhc['Apell'].str[:5]).str.upper()
C:\Users\Leja\AppData\Local\Temp/ipykernel_18304/3476419925.py:12: SettingWithCopyWarning: 
A value is trying to b

In [9]:
####################QUOTES###########################
all_data = pd.DataFrame()

for f in glob.glob(r"C:\Users\Leja\Downloads\MVR\Quotes\*.xlsx"):
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)

#print(all_data)

quotes= all_data.fillna('')

quotes= quotes.reset_index()

quotes.info()

#print(quotes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89349 entries, 0 to 89348
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   index                      89349 non-null  int64         
 1   Applicant ID               89349 non-null  int64         
 2   Applicant Name             89349 non-null  object        
 3   Assigned Agent             89349 non-null  object        
 4   Branch Name                89349 non-null  object        
 5   Email - Primary            89349 non-null  object        
 6   Phone - Home               89349 non-null  object        
 7   Master Company             89349 non-null  object        
 8   Quote Created Date         89349 non-null  object        
 9   Quote ID                   89349 non-null  object        
 10  Quote Premium              89349 non-null  object        
 11  Quote Source               89349 non-null  object        
 12  Quot

In [10]:
quotes['Nombre_'] = quotes['Applicant Name'].apply(lambda s: s.split(',')[-1])
quotes['Nombre_'] = quotes['Nombre_'].str.lstrip()
quotes['Nombre'] = quotes['Nombre_'].apply(lambda s: s.split(' ')[0])
quotes['Apellido'] = quotes['Applicant Name'].apply(lambda s: s.split(',')[0])
quotes['Apellido'] = quotes['Apellido'].apply(lambda s: s.split(' ')[0])
#s.str.upper()

quotes['Name'] = (quotes['Nombre'] + " " + quotes['Apellido']).str.upper()

quotes['Name_2'] = quotes['Name']

quotes['Date'] = pd.to_datetime(quotes['Date Created']).dt.date

quotes['nk'] = (quotes['Nombre'].str[:5] + quotes['Apellido'].str[:5]).str.upper()


#quotes.dtypes
#print(quotes)

In [11]:
quotes_= quotes[['Applicant ID', 'Quoted By', 'Name']]

for i in quotes_['Name'].to_list():
    lhc.loc[lhc['Name'].astype(str).str.contains(i), 'Name' ] = i

MVR_lhc= pd.merge(lhc, quotes_, how="left", on='Name' ).drop_duplicates(subset=['Marca'], keep='first')

MVR_lhc1= MVR_lhc[MVR_lhc['Applicant ID'].notnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que cruzaron

MVR_pend= MVR_lhc[MVR_lhc['Applicant ID'].isnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que no cruzaron

MVR_pend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 17 to 3822
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          106 non-null    object 
 1   Product            106 non-null    object 
 2   Quote Num          106 non-null    object 
 3   Drivers Name       106 non-null    object 
 4   Drivers License #  106 non-null    object 
 5   DL State           106 non-null    object 
 6   Order Date         106 non-null    object 
 7   TransType          106 non-null    object 
 8   Amount             106 non-null    float64
 9   Agent              106 non-null    object 
 10  Quoting Producer   106 non-null    object 
 11  Marca              106 non-null    int64  
 12  nombres            106 non-null    int64  
 13  Nombre             106 non-null    object 
 14  Apell              106 non-null    object 
 15  nk                 106 non-null    object 
 16  Name               106 n

In [12]:
quotes_= quotes[['Applicant ID', 'Quoted By', 'nk']]

#MVR_mar cruza y de ahí me quedo con los cruzados y saco los pendientes
MVR_lhc2 = pd.merge(MVR_pend, quotes_, how="left", on=["nk"]).drop_duplicates(subset=['Marca'], keep='first')

MVR_lhc3= MVR_lhc2[MVR_lhc2['Applicant ID'].notnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que cruzaron
MVR_pend= MVR_lhc2[MVR_lhc2['Applicant ID'].isnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que no cruzaron

MVR_pend.info()   


<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 24 to 499
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          80 non-null     object 
 1   Product            80 non-null     object 
 2   Quote Num          80 non-null     object 
 3   Drivers Name       80 non-null     object 
 4   Drivers License #  80 non-null     object 
 5   DL State           80 non-null     object 
 6   Order Date         80 non-null     object 
 7   TransType          80 non-null     object 
 8   Amount             80 non-null     float64
 9   Agent              80 non-null     object 
 10  Quoting Producer   80 non-null     object 
 11  Marca              80 non-null     int64  
 12  nombres            80 non-null     int64  
 13  Nombre             80 non-null     object 
 14  Apell              80 non-null     object 
 15  nk                 80 non-null     object 
 16  Name               80 non-

In [13]:
#################### L E A D S ###########################
all_data = pd.DataFrame()

for f in glob.glob(r"C:\Users\Leja\Downloads\MVR\leads\*.xlsx"):
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)

#print(all_data)

leads= all_data.fillna('')

leads= leads.reset_index()

leads.info()

#leads=pd.read_excel(r"C:\Users\Leja\Downloads\MVR\leads\Applicant_LeadsDetail 010121 121822.xlsx", sheet_name='Detail')

leads['Last Name'] = leads['Last Name'].fillna("")
leads['Apellido'] = leads['Last Name'].apply(lambda s: s.split(' ')[0])

leads['Name'] = (leads['First Name'] + " " + leads['Last Name']).str.upper()

leads['nk'] = (leads['First Name'].str[:5] + leads['Last Name'].str[:5]).str.upper()

leads['Account Name'] = leads['Account Name'].str.upper()

leads1=leads[['Applicant ID', 'Assigned Agent', 'Account Name']]

#print(leads)
#leads1.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27526 entries, 0 to 27525
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   index                27526 non-null  int64         
 1   Branch Name          27526 non-null  object        
 2   Applicant ID         27526 non-null  int64         
 3   Account Name         27526 non-null  object        
 4   Assigned Agent       27526 non-null  object        
 5   Email  - Primary     27526 non-null  object        
 6   Phone - Home         27526 non-null  object        
 7   Relation             27526 non-null  object        
 8   Account Type         27526 non-null  object        
 9   Applicant Type       27526 non-null  object        
 10  Email - Alternate    27526 non-null  object        
 11  Email - Business     27526 non-null  object        
 12  First Name           27526 non-null  object        
 13  Last Name            27526 non-

In [14]:
for i in leads1['Account Name'].to_list():
    MVR_pend.loc[MVR_pend['Name'].astype(str).str.contains(i), 'Account Name' ] = i

MVR_lhc4= pd.merge(MVR_pend, leads1, how="left", on='Account Name' ).drop_duplicates(subset=['Marca'], keep='first')

MVR_lhc5= MVR_lhc4[MVR_lhc4['Applicant ID'].notnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que cruzaron

MVR_pend= MVR_lhc4[MVR_lhc4['Applicant ID'].isnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que no cruzaron

MVR_pend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 0 to 82
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          45 non-null     object 
 1   Product            45 non-null     object 
 2   Quote Num          45 non-null     object 
 3   Drivers Name       45 non-null     object 
 4   Drivers License #  45 non-null     object 
 5   DL State           45 non-null     object 
 6   Order Date         45 non-null     object 
 7   TransType          45 non-null     object 
 8   Amount             45 non-null     float64
 9   Agent              45 non-null     object 
 10  Quoting Producer   45 non-null     object 
 11  Marca              45 non-null     int64  
 12  nombres            45 non-null     int64  
 13  Nombre             45 non-null     object 
 14  Apell              45 non-null     object 
 15  nk                 45 non-null     object 
 16  Name               45 non-nu

In [15]:
coap=pd.read_excel(r"C:\Users\Leja\Downloads\MVR\leads\Applicant_Detail 010123 062223.xlsx", sheet_name='Detail')

In [16]:
coap['Last Name'] = coap['Last Name'].fillna("")
coap['Apellido'] = coap['Last Name'].apply(lambda s: s.split(' ')[0])

coap['Name'] = (coap['First Name'] + " " + coap['Apellido']).fillna(coap['Account Name'].fillna('')).str.upper()

coap['nk'] = (coap['First Name'].str[:5]).str.upper() + (coap['Apellido'].str[:5]).str.upper()

coap['nk_'] = (coap['First Name'].str[:5]).str.upper() + (coap['Apellido'].str[:5]).fillna('Account Name').str.upper().str.upper()

coap['Account Name']=coap['Account Name'].fillna('Name').str.upper()

coap1= coap[['Applicant ID', 'Assigned Agent', 'Account Name']]
#print(coap)
#coap.dtypes

In [17]:
for i in coap1['Account Name'].to_list():
    MVR_pend.loc[MVR_pend['Name'].astype(str).str.contains(i), 'Account Name' ] = i

MVR_lhc6= pd.merge(MVR_pend, coap1, how="left", on='Account Name' ).drop_duplicates(subset=['Marca'], keep='first')

MVR_lhc7= MVR_lhc6[MVR_lhc6['Applicant ID'].notnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que cruzaron

MVR_pend= MVR_lhc6[MVR_lhc6['Applicant ID'].isnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que no cruzaron

#print(MVR_pend) 

MVR_pend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 0 to 44
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          45 non-null     object 
 1   Product            45 non-null     object 
 2   Quote Num          45 non-null     object 
 3   Drivers Name       45 non-null     object 
 4   Drivers License #  45 non-null     object 
 5   DL State           45 non-null     object 
 6   Order Date         45 non-null     object 
 7   TransType          45 non-null     object 
 8   Amount             45 non-null     float64
 9   Agent              45 non-null     object 
 10  Quoting Producer   45 non-null     object 
 11  Marca              45 non-null     int64  
 12  nombres            45 non-null     int64  
 13  Nombre             45 non-null     object 
 14  Apell              45 non-null     object 
 15  nk                 45 non-null     object 
 16  Name               45 non-nu

In [18]:
coap1= coap[['Applicant ID', 'Assigned Agent', 'nk']]

#MVR_mar cruza y de ahí me quedo con los cruzados y saco los pendientes
MVR_lhc8 = pd.merge(MVR_pend, coap1, how="left", on=["nk"]).drop_duplicates(subset=['Marca'], keep='first')

MVR_lhc9= MVR_lhc8[MVR_lhc8['Applicant ID'].notnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que cruzaron
MVR_pend= MVR_lhc8[MVR_lhc8['Applicant ID'].isnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que no cruzaron

MVR_pend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 0 to 44
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          43 non-null     object 
 1   Product            43 non-null     object 
 2   Quote Num          43 non-null     object 
 3   Drivers Name       43 non-null     object 
 4   Drivers License #  43 non-null     object 
 5   DL State           43 non-null     object 
 6   Order Date         43 non-null     object 
 7   TransType          43 non-null     object 
 8   Amount             43 non-null     float64
 9   Agent              43 non-null     object 
 10  Quoting Producer   43 non-null     object 
 11  Marca              43 non-null     int64  
 12  nombres            43 non-null     int64  
 13  Nombre             43 non-null     object 
 14  Apell              43 non-null     object 
 15  nk                 43 non-null     object 
 16  Name               43 non-nu

In [19]:
######### IMPORTO EL BOOK OF BUSINESS COMPLETODESDE 2018 ESTÁ ESTÉ ARCHIVO SE PUEDE IMPORTAR OTROS DE AHORA EN ADELANTE
bob=pd.read_excel(r"C:\Users\Leja\Downloads\bob status\BOB_all_status 071623.xlsx", sheet_name='Detail')

In [20]:
#bob= bob.loc[bob['Applicant Status']!='Inactive Client']
bob['Assigned Agent']= bob['Assigned Producer']

bob['Account Name'] = bob['Account Name'].fillna(" ")

bob['Name'] = (bob['First Name'] + " " + bob['Last Name']).str.upper()

bob['nk'] = (bob['First Name'].str[:5] + bob['Last Name'].str[:5]).str.upper()

bob1=bob[['Applicant ID', 'Assigned Agent', 'Name']]

#print(bob1)
bob1.dtypes

Applicant ID       int64
Assigned Agent    object
Name              object
dtype: object

In [21]:
for i in bob['Account Name'].to_list():
    MVR_pend.loc[MVR_pend['Name'].astype(str).str.contains(i), 'Account Name' ] = i
    
MVR_lhc10= pd.merge(MVR_pend, bob1, how="left", on='Name' ).drop_duplicates(subset=['Marca'], keep='first')

MVR_lhc11= MVR_lhc10[MVR_lhc10['Applicant ID'].notnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que cruzaron

MVR_pend= MVR_lhc10[MVR_lhc10['Applicant ID'].isnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que no cruzaron

#print(MVR_pend)    

MVR_pend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 6 to 94
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          35 non-null     object 
 1   Product            35 non-null     object 
 2   Quote Num          35 non-null     object 
 3   Drivers Name       35 non-null     object 
 4   Drivers License #  35 non-null     object 
 5   DL State           35 non-null     object 
 6   Order Date         35 non-null     object 
 7   TransType          35 non-null     object 
 8   Amount             35 non-null     float64
 9   Agent              35 non-null     object 
 10  Quoting Producer   35 non-null     object 
 11  Marca              35 non-null     int64  
 12  nombres            35 non-null     int64  
 13  Nombre             35 non-null     object 
 14  Apell              35 non-null     object 
 15  nk                 35 non-null     object 
 16  Name               35 non-nu

In [22]:
bob1=bob[['Applicant ID', 'Assigned Agent', 'nk']]

#cruzar con BOB
MVR_lhc12= pd.merge(MVR_pend, bob1, how="left", left_on=["nk"], right_on=["nk"]).drop_duplicates(subset=['Marca'], keep='last')

#me quedo con los que cruzaron
MVR_lhc13= MVR_lhc12[MVR_lhc12['Applicant ID'].notnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que cruzaron
MVR_pend= MVR_lhc12[MVR_lhc12['Applicant ID'].isnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que no cruzaron

MVR_pend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 43
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          29 non-null     object 
 1   Product            29 non-null     object 
 2   Quote Num          29 non-null     object 
 3   Drivers Name       29 non-null     object 
 4   Drivers License #  29 non-null     object 
 5   DL State           29 non-null     object 
 6   Order Date         29 non-null     object 
 7   TransType          29 non-null     object 
 8   Amount             29 non-null     float64
 9   Agent              29 non-null     object 
 10  Quoting Producer   29 non-null     object 
 11  Marca              29 non-null     int64  
 12  nombres            29 non-null     int64  
 13  Nombre             29 non-null     object 
 14  Apell              29 non-null     object 
 15  nk                 29 non-null     object 
 16  Name               29 non-nu

In [23]:
#Combina todos los MVR:
MVR_NG = pd.concat([MVR_lhc1, MVR_lhc3, MVR_lhc5, MVR_lhc7, MVR_lhc9, MVR_lhc11, MVR_lhc13], sort=False, ignore_index=True, axis=0) 

MVR_NG = MVR_NG.drop_duplicates(subset=['Marca'], keep='last')

MVR_NG.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          300 non-null    object 
 1   Product            300 non-null    object 
 2   Quote Num          300 non-null    object 
 3   Drivers Name       300 non-null    object 
 4   Drivers License #  300 non-null    object 
 5   DL State           300 non-null    object 
 6   Order Date         300 non-null    object 
 7   TransType          300 non-null    object 
 8   Amount             300 non-null    float64
 9   Agent              300 non-null    object 
 10  Quoting Producer   300 non-null    object 
 11  Marca              300 non-null    int64  
 12  nombres            300 non-null    int64  
 13  Nombre             300 non-null    object 
 14  Apell              300 non-null    object 
 15  nk                 300 non-null    object 
 16  Name               300 non

In [24]:
#MVR_NG.to_excel(r'C:\Users\Leja\Downloads\MVR_NG.xlsx', index=False)

In [25]:
MVR_NG_ = MVR_NG[['Quote Num', 'Account Name', 'Applicant ID', 'Assigned Agent', 'Quoted By']]

MVR_lhc14 = pd.merge(MVR_pend, MVR_NG_, how="left", on=['Quote Num']).drop_duplicates(subset=['Marca'], keep='first')

#me quedo con los que cruzaron
MVR_lhc15= MVR_lhc14[MVR_lhc14['Applicant ID'].notnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que cruzaron
MVR_pend= MVR_lhc14[MVR_lhc14['Applicant ID'].isnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que no cruzaron

MVR_pend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 28
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          29 non-null     object 
 1   Product            29 non-null     object 
 2   Quote Num          29 non-null     object 
 3   Drivers Name       29 non-null     object 
 4   Drivers License #  29 non-null     object 
 5   DL State           29 non-null     object 
 6   Order Date         29 non-null     object 
 7   TransType          29 non-null     object 
 8   Amount             29 non-null     float64
 9   Agent              29 non-null     object 
 10  Quoting Producer   29 non-null     object 
 11  Marca              29 non-null     int64  
 12  nombres            29 non-null     int64  
 13  Nombre             29 non-null     object 
 14  Apell              29 non-null     object 
 15  nk                 29 non-null     object 
 16  Name               29 non-nu

In [26]:
#Combina todos los MVR:
MVR_NG = pd.concat([MVR_NG, MVR_lhc14], sort=False, ignore_index=True, axis=0) 
#MVR_Prog = MVR_Prog.drop_duplicates(subset=['Marca'], keep='last')

MVR_NG = MVR_NG.drop_duplicates(subset=['Marca'], keep='first')
MVR_NG.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 0 to 328
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          329 non-null    object 
 1   Product            329 non-null    object 
 2   Quote Num          329 non-null    object 
 3   Drivers Name       329 non-null    object 
 4   Drivers License #  329 non-null    object 
 5   DL State           329 non-null    object 
 6   Order Date         329 non-null    object 
 7   TransType          329 non-null    object 
 8   Amount             329 non-null    float64
 9   Agent              329 non-null    object 
 10  Quoting Producer   329 non-null    object 
 11  Marca              329 non-null    int64  
 12  nombres            329 non-null    int64  
 13  Nombre             329 non-null    object 
 14  Apell              329 non-null    object 
 15  nk                 329 non-null    object 
 16  Name               329 non

In [27]:
#vhc= df[df['TransType']=='Viol Hist Chargeback']
MVR_NG_ = MVR_NG[['Quote Num', 'Account Name', 'Applicant ID', 'Assigned Agent', 'Quoted By']]

MVR_lhc16 = pd.merge(vhc, MVR_NG_, how="left", on=['Quote Num']).drop_duplicates(subset=['Marca'], keep='first')
#me quedo con los que no cruzaron
MVR_pend= MVR_lhc16[MVR_lhc16['Applicant ID'].isnull()].dropna(axis = 1, how = 'all') #se queda sólo con los que no cruzaron

MVR_NG_1 = pd.concat([MVR_NG, MVR_lhc16], sort=False, ignore_index=True, axis=0)


MVR_pend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 6 to 184
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          25 non-null     object 
 1   Product            25 non-null     object 
 2   Quote Num          25 non-null     object 
 3   Drivers Name       25 non-null     object 
 4   Drivers License #  25 non-null     object 
 5   DL State           25 non-null     object 
 6   Order Date         25 non-null     object 
 7   TransType          25 non-null     object 
 8   Amount             25 non-null     float64
 9   Agent              25 non-null     object 
 10  Quoting Producer   25 non-null     object 
 11  Marca              25 non-null     int64  
dtypes: float64(1), int64(1), object(10)
memory usage: 2.5+ KB


In [28]:
MVR_NG_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514 entries, 0 to 513
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gov State          514 non-null    object 
 1   Product            514 non-null    object 
 2   Quote Num          514 non-null    object 
 3   Drivers Name       514 non-null    object 
 4   Drivers License #  514 non-null    object 
 5   DL State           514 non-null    object 
 6   Order Date         514 non-null    object 
 7   TransType          514 non-null    object 
 8   Amount             514 non-null    float64
 9   Agent              514 non-null    object 
 10  Quoting Producer   514 non-null    object 
 11  Marca              514 non-null    int64  
 12  nombres            329 non-null    float64
 13  Nombre             329 non-null    object 
 14  Apell              329 non-null    object 
 15  nk                 329 non-null    object 
 16  Name               329 non

In [29]:
MVR_NG1= MVR_NG_1[['Quoting Producer']].drop_duplicates()

MVR_NG1['QB']= MVR_NG1['Quoting Producer'].apply(lambda s: s.split(' ')[0])

MVR_NG2= quotes_[['Quoted By']].drop_duplicates()

MVR_NG2['QB']= MVR_NG2['Quoted By'].apply(lambda s: s.split(', ')[1])

#.fillna(MVR_NG1['Quoting Producer'].apply(lambda s: s.split(' ')[1]))

MVR_NG3= pd.merge(MVR_NG2, MVR_NG1, how="outer", on='QB' ).drop_duplicates(subset=['QB'], keep='first')

print(MVR_NG3)

                                      Quoted By                QB  \
0                            Cervantes, Giselle           Giselle   
1                              Romero, Elisabet          Elisabet   
2                              Puentes, Adriana           Adriana   
4                       Renewal Team , valeria           valeria    
5                              Freay, Alejandro         Alejandro   
6                               Garcia, Genesis           Genesis   
7                                   Lora, Luisa             Luisa   
8                                  Karen, Karen             Karen   
9                             DeLaRosa, Yadiris           Yadiris   
10                               Da Costa, Joel              Joel   
12            of Victor Dueri, Paola assistant   Paola assistant    
13                              Puentes, Eliana            Eliana   
15                             Arroyave, Steven            Steven   
16                          Dacost

In [30]:
MVR_NG_2= pd.merge(MVR_NG_1, MVR_NG3, how="left", on='Quoting Producer' ).drop_duplicates(subset=['Marca'], keep='first')

print(MVR_NG)

    Gov State Product   Quote Num              Drivers Name Drivers License #  \
0          NC     PPA  98341425.0  NICOLE CALIHUA HERNANDEZ                     
1          NC     PPA  98340786.0  ANDERSON MENENDEZ CAMPOS                     
2          NC     PPA  98250425.0     ALEXIS ANDRADE OLIVAS                     
3          NC     PPA  98249316.0              NEILU RINCON                     
4          NC     PPA  98231233.0  BELARMINO GARCIA SERRANO                     
..        ...     ...         ...                       ...               ...   
324        FL     PPA  99858221.0              YESICA PLAZA                     
325        FL     PPA  99623323.0          JONATHAN OQUENDO                     
326        FL      CV  99110006.0          CAROLINE VULPINI                     
327        FL      CV  99121944.0          CAROLINE VULPINI                     
328        FL      CV  99124751.0          CAROLINE VULPINI                     

    DL State Order Date    

In [31]:
#MVR_NG_2['Quoting Producer_x']= MVR_NG_2['Quoting Producer_y']

l=[('Ingrid', 'Jose', 'Sandra')]

MVR_NG_2['QBy']= MVR_NG_2['Quoted By_y'].fillna(MVR_NG_2['Quoted By_x'])

MVR_NG_2['Quoted By']= np.where((MVR_NG_2['QB'].isin(l)), MVR_NG_2['QBy'].fillna(MVR_NG_2['Assigned Agent']), MVR_NG_2['Assigned Agent'].fillna(MVR_NG_2['QBy']))

MVR_NG_2['Applicant ID']= MVR_NG_2['Applicant ID'].fillna('No identificado')

#np.where(df.period.str[:3].isin(l),'month',df.period.str.split(' ').str[0])
MVR_NG_2.head()

,Gov State,Product,Quote Num,Drivers Name,Drivers License #,DL State,Order Date,TransType,Amount,Agent,...,nk,Name,Applicant ID,Quoted By_x,Account Name,Assigned Agent,Quoted By_y,QB,QBy,Quoted By
0,NC,PPA,98341425.0,NICOLE CALIHUA HERNANDEZ,,NC,4/4/2023,Loss Hist Chargeback,3.5,53537.0,...,NICOLCALIH,NICOLE CALIHUA,121041127.0,"Lora, Luisa",NaN,NaN,"Valbuena, Ingrid",Ingrid,"Valbuena, Ingrid","Valbuena, Ingrid"
1,NC,PPA,98340786.0,ANDERSON MENENDEZ CAMPOS,,NC,4/4/2023,Loss Hist Chargeback,3.5,53537.0,...,ANDERMENEN,ANDERSON MENENDEZ,132676079.0,"Lora, Luisa",NaN,NaN,"Valbuena, Ingrid",Ingrid,"Valbuena, Ingrid","Valbuena, Ingrid"
2,NC,PPA,98250425.0,ALEXIS ANDRADE OLIVAS,,NC,4/3/2023,Loss Hist Chargeback,3.5,53537.0,...,ALEXIANDRA,ALEXIS ANDRADE,43203754.0,"P, Adriana",NaN,NaN,"Valbuena, Ingrid",Ingrid,"Valbuena, Ingrid","Valbuena, Ingrid"
3,NC,PPA,98249316.0,NEILU RINCON,,NC,4/3/2023,Loss Hist Chargeback,3.5,53537.0,...,NEILURINCO,NEILU RINCON,132577667.0,"Lora, Luisa",NaN,NaN,"Valbuena, Ingrid",Ingrid,"Valbuena, Ingrid","Valbuena, Ingrid"
4,NC,PPA,98231233.0,BELARMINO GARCIA SERRANO,,NC,4/3/2023,Loss Hist Chargeback,3.5,53537.0,...,BELARGARCI,BELARMINO GARCIA,43203580.0,"Romero, Elisabet",NaN,NaN,"Valbuena, Ingrid",Ingrid,"Valbuena, Ingrid","Valbuena, Ingrid"


In [32]:
MVR_NG_3=MVR_NG_2[['Drivers Name',  'Quote Num', 'Gov State', 'Order Date', 'TransType',  'Amount', 'Applicant ID',	
               'Assigned Agent', 'Quoted By', 'Quoted By_x', 'Quoted By_y', 'QB', 'Quoting Producer', 'Agent', 'Marca']]

#print(MVR_NG)
#MVR_NG.dtypes

MVR_NG_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 514 entries, 0 to 513
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Drivers Name      514 non-null    object 
 1   Quote Num         514 non-null    object 
 2   Gov State         514 non-null    object 
 3   Order Date        514 non-null    object 
 4   TransType         514 non-null    object 
 5   Amount            514 non-null    float64
 6   Applicant ID      514 non-null    object 
 7   Assigned Agent    71 non-null     object 
 8   Quoted By         504 non-null    object 
 9   Quoted By_x       389 non-null    object 
 10  Quoted By_y       455 non-null    object 
 11  QB                514 non-null    object 
 12  Quoting Producer  514 non-null    object 
 13  Agent             514 non-null    object 
 14  Marca             514 non-null    int64  
dtypes: float64(1), int64(1), object(13)
memory usage: 64.2+ KB


In [33]:
#Resumen según agente

mvr_agg = pd.pivot_table(MVR_NG_3, values='Amount', index=['Quoted By'], aggfunc='sum', fill_value=0)
mvr_agg_ = pd.pivot_table(MVR_NG_3, values='Applicant ID', index=['Quoted By'], aggfunc=lambda x: len(x.unique()), fill_value=0)
mvr_agg_ = mvr_agg_.rename(columns={'Applicant ID':'Clientes'})

mvr_agg__ = pd.pivot_table(MVR_NG_3, values='Applicant ID', index=['Quoted By'], aggfunc='count', fill_value=0)
mvr_agg__ = mvr_agg__.rename(columns={'Applicant ID':'MVR'})

result = pd.concat([mvr_agg, mvr_agg__, mvr_agg_], axis=1, join="inner")
#result['MVR/client']= result['MVR'] / result['Clientes'].apply(lambda x: float("{:.2f}".format(x)))
result_agg = result.append(result.sum().rename('Total'))

mvr_cod = pd.pivot_table(MVR_NG_3, values='Amount', index=['Agent'], aggfunc='sum', fill_value=0)


print(result_agg)
print(mvr_cod)


                           Amount    MVR  Clientes
Quoted By                                         
Acevedo, Carlos             28.02    4.0       2.0
Arroyave, Steven            82.60   15.0       7.0
Caruso, Rosana              26.50    3.0       1.0
Cervantes, Giselle         416.18   76.0      49.0
Dacosta, Betsy             144.02   28.0      21.0
DeLaRosa, Yadiris          196.00   24.0      10.0
Dueri, Victor              138.20   20.0       8.0
Freay, Alejandro            35.56    5.0       2.0
Frontdesk, Gabi              3.50    1.0       1.0
Garcia, Genesis            193.52   30.0      18.0
Health, Metropolitan Ins    24.50    4.0       2.0
Karen, Karen                32.50    7.0       6.0
Lo, Harin                   53.00    6.0       1.0
Lora, Luisa                 64.00   16.0      13.0
Puentes, Adriana            15.00    2.0       1.0
Renewal Team , valeria      10.50    3.0       3.0
Renewal Team, Joany         26.50    3.0       1.0
Rodriquez, Daniela          22.

In [34]:
writer = pd.ExcelWriter(r'C:\Users\Leja\Downloads\MVR\MVR\MVR_NG_.xlsx', engine='xlsxwriter')

MVR_NG_3= MVR_NG_3[MVR_NG_3['Amount']> 0]

# Write each dataframe to a different worksheet.
#EXPIRACIONES
MVR_NG_3.to_excel(writer, sheet_name='MVR_ng', index=False)
#MVR_pend.to_excel(writer, sheet_name='pend', index=False)
#result.to_excel(writer, sheet_name='resumen')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [35]:
#MVR_NG= MVR_NG[MVR_NG['Amount']> 0]

#uniques = MVR_NG['Quoted By'].unique()

#for i in uniques:
#    with pd.ExcelWriter(f'{i} MVR National.xlsx') as writer:
#        MVR_NG[MVR_NG['Quoted By'] == i].to_excel(writer, index=None, sheet_name='Quoted by')